In [109]:
import networkx as nx
import random
import numpy as np
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

In [110]:
def generate_graph(number_of_nodes):
    G = nx.DiGraph()

    # Adding nodes
    G.add_nodes_from(range(number_of_nodes))

    # Adding edges with random weights between 1 and 6
    for i in range(number_of_nodes - 1):
        for j in range(i+1, number_of_nodes):
            if random.random() < 0.3:
                weight = random.randint(1, 6)
                G.add_edge(i, j, weight=weight)
                
    longest_path = nx.dag_longest_path(G, weight='weight')
    
    return [G, longest_path]

In [111]:
def graph_to_distance_matrix(G):
    number_of_nodes = G.number_of_nodes()
    matrix = [[0 for _ in range(number_of_nodes)] for _ in range(number_of_nodes)]

    for i in range(number_of_nodes - 1):
        for j in range(i + 1, number_of_nodes):
            if G.has_edge(i, j):
                matrix[i][j] = G[i][j]['weight']
            else:
                matrix[i][j] = 0
                matrix[j][i] = 0

    return matrix

In [112]:
def fitness_function(positions, distance_matrix):
    costs = []

    for position in positions:
        path = [int(p) for p in np.round(position)]
        path = list(dict.fromkeys(path))

        if len(path) < 2:
            costs.append(-1e8)
            continue

        path_cost = 0
        for i in range(len(path) - 1):
            path_cost += distance_matrix[path[i]][path[i + 1]]

        costs.append(path_cost)

    return np.array(costs)


In [113]:
# Generate graph and ideal solution
G, ideal_longest_path = generate_graph(10)

# Convert graph to distance matrix
distance_matrix = graph_to_distance_matrix(G)

In [114]:
# Particle swarm optimization
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.9}
n_particles = 200
n_iterations = 500
constraints = (np.zeros(G.number_of_nodes()), (G.number_of_nodes() - 1) * np.ones(G.number_of_nodes()))
# optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=G.number_of_nodes(), options=options, bounds=constraints)
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=G.number_of_nodes(), options=options, bounds=constraints)
# optimizer = ps.single.LocalBestPSO(n_particles=n_particles, dimensions=G.number_of_nodes(), options=options, bounds=constraints)
best_cost, best_pos = optimizer.optimize(lambda x: -fitness_function(x, distance_matrix), iters=n_iterations)

2023-04-04 11:51:20,538 - pyswarms.single.global_best - INFO - Optimize for 500 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|500/500, best_cost=-19
2023-04-04 11:51:22,367 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -19.0, best pos: [5.55786083 8.45188155 6.17608126 1.6534039  3.98554835 2.86274008
 8.31584828 4.88455881 1.81448258 8.94959215]


In [115]:
# Extract the longest path found by PSO
longest_path_pso = [int(p) for p in np.round(best_pos)]
longest_path_pso = list(dict.fromkeys(longest_path_pso))

In [116]:
# Compare results
print("Ideal longest path:", ideal_longest_path)
print("PSO longest path:", longest_path_pso)

Ideal longest path: [1, 3, 5, 7, 8]
PSO longest path: [6, 8, 2, 4, 3, 5, 9]
